In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import psycopg2
import glob
import imageio
import matplotlib.pyplot as plt
import os
import PIL
from tensorflow.keras import callbacks
from tensorflow import keras
from tensorflow.keras import layers
import time
from IPython import display
import tensorflow as tf
import seaborn as sns
from PIL import Image
import io
# import pickle5 as pickle
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2021-11-14 06:40:40.359845: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


/kaggle/input/ventilator-pressure-prediction/sample_submission.csv
/kaggle/input/ventilator-pressure-prediction/train.csv
/kaggle/input/ventilator-pressure-prediction/test.csv


In [2]:
df_train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv', index_col = 'id')
X_test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv', index_col = 'id')
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

X_train = df_train.copy().drop('pressure', axis = 1)
Y_train = df_train['pressure'].copy()

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
X_test

,breath_id,R,C,time_step,u_in,u_out
id,,,,,,
1,0,5,20,0.000000,0.000000,0
2,0,5,20,0.031904,7.515046,0
3,0,5,20,0.063827,14.651675,0
4,0,5,20,0.095751,21.230610,0
5,0,5,20,0.127644,26.320956,0
...,...,...,...,...,...,...
4023996,125748,20,10,2.530117,4.971245,1
4023997,125748,20,10,2.563853,4.975709,1
4023998,125748,20,10,2.597475,4.979468,1


# My Test Example

In [4]:
# # Define X and Y
X_train = X_train.iloc[:80000]
Y_train = Y_train.iloc[:80000]

In [5]:
# Drop the u_out = 1 rows since the model will not be graded on those rows, but leave them in for X_test since we still want the full number of predictions
# X_train = X_train[X_train['u_out'] == 0]


In [6]:
# X_train.drop(axis=1, columns=['breath_id'], inplace=True)
X_test.drop(axis=1, columns=['breath_id'], inplace=True)

In [7]:
X_train

,breath_id,R,C,time_step,u_in,u_out
id,,,,,,
1,1,20,50,0.000000,0.083334,0
2,1,20,50,0.033652,18.383041,0
3,1,20,50,0.067514,22.509278,0
4,1,20,50,0.101542,22.808822,0
5,1,20,50,0.135756,25.355850,0
...,...,...,...,...,...,...
79996,1612,50,20,2.395774,4.944552,1
79997,1612,50,20,2.427758,4.952747,1
79998,1612,50,20,2.459710,4.959724,1


# Feature Engineering

In [8]:
# Didn't improve the model, so I'm skipping these for now
X_train['u_in_cumsum'] = (X_train['u_in']).groupby(X_train['breath_id']).cumsum()
X_test['u_in_cumsum'] = (X_test['u_in']).groupby(X_test['breath_id']).cumsum()
X_train['area'] = X_train['time_step'] * X_train['u_in']
X_train['area'] = X_train.groupby('breath_id')['area'].cumsum()
X_test['area'] = X_test['time_step'] * X_test['u_in']
X_test['area'] = X_test.groupby('breath_id')['area'].cumsum()
print(len(X_train))
X_train['cross']= X_train['u_in']*X_train['u_out']
X_train['cross2']= X_train['time_step']*X_train['u_out']
X_test['cross']= X_test['u_in']*X_test['u_out']
X_test['cross2']= X_test['time_step']*X_test['u_out']
print(len(X_train))

X_train['one'] = 1
X_train['count'] = (X_train['one']).groupby(X_train['breath_id']).cumsum()
X_train['u_in_cummean'] =X_train['u_in_cumsum'] /X_train['count']
X_test['one'] = 1
X_test['count'] = (X_test['one']).groupby(X_test['breath_id']).cumsum()
X_test['u_in_cummean'] =X_test['u_in_cumsum'] /X_test['count']
print(len(X_train))

X_train['breath_id_lag']=X_train['breath_id'].shift(1).fillna(0)
X_train['breath_id_lag2']=X_train['breath_id'].shift(2).fillna(0)
X_train['breath_id_lagsame']=np.select([X_train['breath_id_lag']==X_train['breath_id']],[1],0)
X_train['breath_id_lag2same']=np.select([X_train['breath_id_lag2']==X_train['breath_id']],[1],0)
X_train['u_in_lag'] = X_train['u_in'].shift(1).fillna(0)
X_train['u_in_lag'] = X_train['u_in_lag']*X_train['breath_id_lagsame']
X_train['u_in_lag2'] = X_train['u_in'].shift(2).fillna(0)
X_train['u_in_lag2'] = X_train['u_in_lag2']*X_train['breath_id_lag2same']
X_train['u_out_lag2'] = X_train['u_out'].shift(2).fillna(0)
X_train['u_out_lag2'] = X_train['u_out_lag2']*X_train['breath_id_lag2same']
print(len(X_train))

X_test['breath_id_lag']=X_test['breath_id'].shift(1).fillna(0)
X_test['breath_id_lag2']=X_test['breath_id'].shift(2).fillna(0)
X_test['breath_id_lagsame']=np.select([X_test['breath_id_lag']==X_test['breath_id']],[1],0)
X_test['breath_id_lag2same']=np.select([X_test['breath_id_lag2']==X_test['breath_id']],[1],0)
X_test['u_in_lag'] = X_test['u_in'].shift(1).fillna(0)
X_test['u_in_lag'] = X_test['u_in_lag']*X_test['breath_id_lagsame']
X_test['u_in_lag2'] = X_test['u_in'].shift(2).fillna(0)
X_test['u_in_lag2'] = X_test['u_in_lag2']*X_test['breath_id_lag2same']
X_test['u_out_lag2'] = X_test['u_out'].shift(2).fillna(0)
X_test['u_out_lag2'] = X_test['u_out_lag2']*X_test['breath_id_lag2same']
print(len(X_train))


#Lag features. 
#Number of lag features inspired by https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm
# X_train['u_in_lag1_'] = X_train.groupby('breath_id')['u_in'].shift(1).fillna(0)
# X_test['u_in_lag1_'] = X_test.groupby('breath_id')['u_in'].shift(1).fillna(0)
# X_train['u_in_lag2_'] = X_train.groupby('breath_id')['u_in'].shift(2).fillna(0)
# X_test['u_in_lag2_'] = X_test.groupby('breath_id')['u_in'].shift(2).fillna(0)
# X_train['u_in_lag3_'] = X_train.groupby('breath_id')['u_in'].shift(3).fillna(0)
# X_test['u_in_lag3_'] = X_test.groupby('breath_id')['u_in'].shift(3).fillna(0)
# X_train['u_in_lag4_'] = X_train.groupby('breath_id')['u_in'].shift(4).fillna(0)
# X_test['u_in_lag4_'] = X_test.groupby('breath_id')['u_in'].shift(4).fillna(0)
print(len(X_train))

X_train['u_in_lag-1'] = X_train.groupby('breath_id')['u_in'].shift(-1).fillna(0)
X_test['u_in_lag-1'] = X_test.groupby('breath_id')['u_in'].shift(-1).fillna(0)
X_train['u_in_lag-2'] = X_train.groupby('breath_id')['u_in'].shift(-2).fillna(0)
X_test['u_in_lag-2'] = X_test.groupby('breath_id')['u_in'].shift(-2).fillna(0)
X_train['u_in_lag-3'] = X_train.groupby('breath_id')['u_in'].shift(-3).fillna(0)
X_test['u_in_lag-3'] = X_test.groupby('breath_id')['u_in'].shift(-3).fillna(0)
X_train['u_in_lag-4'] = X_train.groupby('breath_id')['u_in'].shift(-4).fillna(0)
X_test['u_in_lag-4'] = X_test.groupby('breath_id')['u_in'].shift(-4).fillna(0)

X_train['u_out_lag-1'] = X_train.groupby('breath_id')['u_out'].shift(-1).fillna(0)
X_test['u_out_lag-1'] = X_test.groupby('breath_id')['u_out'].shift(-1).fillna(0)
X_train['u_out_lag-2'] = X_train.groupby('breath_id')['u_out'].shift(-2).fillna(0)
X_test['u_out_lag-2'] = X_test.groupby('breath_id')['u_out'].shift(-2).fillna(0)
X_train['u_out_lag-3'] = X_train.groupby('breath_id')['u_out'].shift(-3).fillna(0)
X_test['u_out_lag-3'] = X_test.groupby('breath_id')['u_out'].shift(-3).fillna(0)
X_train['u_out_lag-4'] = X_train.groupby('breath_id')['u_out'].shift(-4).fillna(0)
X_test['u_out_lag-4'] = X_test.groupby('breath_id')['u_out'].shift(-4).fillna(0)
print(len(X_train))

#Window features
#Choice of features inspired vy https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm
X_train['breath_id__u_in__max'] = X_train.groupby(['breath_id'])['u_in'].transform('max')
X_test['breath_id__u_in__max'] = X_test.groupby(['breath_id'])['u_in'].transform('max')
X_train['breath_id__u_in__mean'] = X_train.groupby(['breath_id'])['u_in'].transform('mean')
X_test['breath_id__u_in__mean'] = X_test.groupby(['breath_id'])['u_in'].transform('mean')
print(len(X_train))

X_train['breath_id__u_out__max'] = X_train.groupby(['breath_id'])['u_out'].transform('max')
X_test['breath_id__u_out__max'] = X_test.groupby(['breath_id'])['u_out'].transform('max')
X_train['breath_id__u_out__mean'] = X_train.groupby(['breath_id'])['u_out'].transform('mean')
X_test['breath_id__u_out__mean'] = X_test.groupby(['breath_id'])['u_out'].transform('mean')

X_train['R'] = X_train['R'].astype(str)
X_train['C'] = X_train['C'].astype(str)
X_train['RC'] = X_train['R']+X_train['C']
X_test['R'] = X_test['R'].astype(str)
X_test['C'] = X_test['C'].astype(str)
X_test['RC'] = X_test['R']+"_"+X_test['C']
print(len(X_train))

X_train = X_train.drop(['R','C','breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same','u_out_lag2'], axis = 1)
X_test = X_test.drop(['R','C','breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same','u_out_lag2'], axis = 1)

print(len(X_train))

X_train

KeyError: 'breath_id'

In [ ]:
# Converts the string fields (RC) to one hot encoded fields
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)


In [ ]:
# Change values to include a masking layer when u_out == 1 since that is not graded in the final model
for i, x in X_train.iterrows():
    if x['u_out'] == 1:
        for column in X_train.columns:
            X_train[column][i] = -1

In [ ]:
X_train[150:200]

In [ ]:
# Reshape the data
X_train = X_train.values.reshape(-1,160,5) # Reshaping into the 3 dimensional tensor that is needed for a recurrent neural network
Y_train = Y_train.values.reshape(-1,160,1)
# Only needed if feature engineering is used
# X_train = X_train.values.reshape(-1,160,31) # Reshaping into the 3 dimensional tensor that is needed for a recurrent neural network
# Y_train = Y_train.values.reshape(-1,160,1)


In [ ]:
# Find the dimensions
print(X_train.shape)
Y_train.shape

In [ ]:
# import packages
from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras.metrics import MeanAbsoluteError

# Taken from https://www.kaggle.com/valentinateslenko/ventilator-pressure-tensorflow-feature-engineering
def BidirLSTM(units):
    return layers.Bidirectional(layers.LSTM(units, return_sequences=True))

def BidirGRU(units):
    return layers.Bidirectional(layers.GRU(units, return_sequences=True, dropout=0.00004))

def make_lstm_model():
    return Sequential([
            layers.Input(shape = (160,5)),
            layers.Masking(mask_value = -1),
            BidirGRU(500),
            BidirGRU(400),
            BidirGRU(300),
            BidirGRU(200),
            layers.Dense(200, activation='swish'),
            layers.Dense(1, activation = 'linear')
        ])

In [ ]:
model = make_lstm_model()
model.compile(loss='mae', optimizer = 'adam',
              metrics=[MeanAbsoluteError()])

model.fit(X_train, Y_train,
batch_size = 1000, epochs=100)
# MAE of 2.523 on the public leaderboard with only 80K rows of data
# MAE of 2.8245 with feature engineering on the public leaderboard

In [ ]:
X_test

In [ ]:
# Make the predictions

X_test = X_test.drop(axis=1, columns=['breath_id']) # Don't need to drop if doing the feature engineering
X_test = X_test.values.reshape(-1,160,5)
Y_test = model.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
export = Y_test.reshape(-1,1)

In [ ]:
export

In [ ]:
id_ = list(np.array(list(range(len(export)))) + 1)

In [ ]:
list(np.ndarray.flatten(export))

fin = pd.DataFrame({'id':id_, 'pressure': list(np.ndarray.flatten(export))})

In [ ]:
X_test.shape

In [ ]:
fin

In [ ]:
fin.to_csv('mysubmission.csv', index=False)

In [ ]:
X_train['u_in_cumsum'] = (X_train['u_in']).groupby(X_train['breath_id']).cumsum()
X_test['u_in_cumsum'] = (X_test['u_in']).groupby(X_test['breath_id']).cumsum()
X_train['area'] = X_train['time_step'] * X_train['u_in']
X_train['area'] = X_train.groupby('breath_id')['area'].cumsum()
X_test['area'] = X_test['time_step'] * X_test['u_in']
X_test['area'] = X_test.groupby('breath_id')['area'].cumsum()
print(len(X_train))
X_train['cross']= X_train['u_in']*X_train['u_out']
X_train['cross2']= X_train['time_step']*X_train['u_out']
X_test['cross']= X_test['u_in']*X_test['u_out']
X_test['cross2']= X_test['time_step']*X_test['u_out']
print(len(X_train))

X_train['one'] = 1
X_train['count'] = (X_train['one']).groupby(X_train['breath_id']).cumsum()
X_train['u_in_cummean'] =X_train['u_in_cumsum'] /X_train['count']
X_test['one'] = 1
X_test['count'] = (X_test['one']).groupby(X_test['breath_id']).cumsum()
X_test['u_in_cummean'] =X_test['u_in_cumsum'] /X_test['count']
print(len(X_train))

X_train['breath_id_lag']=X_train['breath_id'].shift(1).fillna(0)
X_train['breath_id_lag2']=X_train['breath_id'].shift(2).fillna(0)
X_train['breath_id_lagsame']=np.select([X_train['breath_id_lag']==X_train['breath_id']],[1],0)
X_train['breath_id_lag2same']=np.select([X_train['breath_id_lag2']==X_train['breath_id']],[1],0)
X_train['u_in_lag'] = X_train['u_in'].shift(1).fillna(0)
X_train['u_in_lag'] = X_train['u_in_lag']*X_train['breath_id_lagsame']
X_train['u_in_lag2'] = X_train['u_in'].shift(2).fillna(0)
X_train['u_in_lag2'] = X_train['u_in_lag2']*X_train['breath_id_lag2same']
X_train['u_out_lag2'] = X_train['u_out'].shift(2).fillna(0)
X_train['u_out_lag2'] = X_train['u_out_lag2']*X_train['breath_id_lag2same']
print(len(X_train))

X_test['breath_id_lag']=X_test['breath_id'].shift(1).fillna(0)
X_test['breath_id_lag2']=X_test['breath_id'].shift(2).fillna(0)
X_test['breath_id_lagsame']=np.select([X_test['breath_id_lag']==X_test['breath_id']],[1],0)
X_test['breath_id_lag2same']=np.select([X_test['breath_id_lag2']==X_test['breath_id']],[1],0)
X_test['u_in_lag'] = X_test['u_in'].shift(1).fillna(0)
X_test['u_in_lag'] = X_test['u_in_lag']*X_test['breath_id_lagsame']
X_test['u_in_lag2'] = X_test['u_in'].shift(2).fillna(0)
X_test['u_in_lag2'] = X_test['u_in_lag2']*X_test['breath_id_lag2same']
X_test['u_out_lag2'] = X_test['u_out'].shift(2).fillna(0)
X_test['u_out_lag2'] = X_test['u_out_lag2']*X_test['breath_id_lag2same']
print(len(X_train))


#Lag features. 
#Number of lag features inspired by https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm
# X_train['u_in_lag1_'] = X_train.groupby('breath_id')['u_in'].shift(1).fillna(0)
# X_test['u_in_lag1_'] = X_test.groupby('breath_id')['u_in'].shift(1).fillna(0)
# X_train['u_in_lag2_'] = X_train.groupby('breath_id')['u_in'].shift(2).fillna(0)
# X_test['u_in_lag2_'] = X_test.groupby('breath_id')['u_in'].shift(2).fillna(0)
# X_train['u_in_lag3_'] = X_train.groupby('breath_id')['u_in'].shift(3).fillna(0)
# X_test['u_in_lag3_'] = X_test.groupby('breath_id')['u_in'].shift(3).fillna(0)
# X_train['u_in_lag4_'] = X_train.groupby('breath_id')['u_in'].shift(4).fillna(0)
# X_test['u_in_lag4_'] = X_test.groupby('breath_id')['u_in'].shift(4).fillna(0)
print(len(X_train))

X_train['u_in_lag-1'] = X_train.groupby('breath_id')['u_in'].shift(-1).fillna(0)
X_test['u_in_lag-1'] = X_test.groupby('breath_id')['u_in'].shift(-1).fillna(0)
X_train['u_in_lag-2'] = X_train.groupby('breath_id')['u_in'].shift(-2).fillna(0)
X_test['u_in_lag-2'] = X_test.groupby('breath_id')['u_in'].shift(-2).fillna(0)
X_train['u_in_lag-3'] = X_train.groupby('breath_id')['u_in'].shift(-3).fillna(0)
X_test['u_in_lag-3'] = X_test.groupby('breath_id')['u_in'].shift(-3).fillna(0)
X_train['u_in_lag-4'] = X_train.groupby('breath_id')['u_in'].shift(-4).fillna(0)
X_test['u_in_lag-4'] = X_test.groupby('breath_id')['u_in'].shift(-4).fillna(0)

X_train['u_out_lag-1'] = X_train.groupby('breath_id')['u_out'].shift(-1).fillna(0)
X_test['u_out_lag-1'] = X_test.groupby('breath_id')['u_out'].shift(-1).fillna(0)
X_train['u_out_lag-2'] = X_train.groupby('breath_id')['u_out'].shift(-2).fillna(0)
X_test['u_out_lag-2'] = X_test.groupby('breath_id')['u_out'].shift(-2).fillna(0)
X_train['u_out_lag-3'] = X_train.groupby('breath_id')['u_out'].shift(-3).fillna(0)
X_test['u_out_lag-3'] = X_test.groupby('breath_id')['u_out'].shift(-3).fillna(0)
X_train['u_out_lag-4'] = X_train.groupby('breath_id')['u_out'].shift(-4).fillna(0)
X_test['u_out_lag-4'] = X_test.groupby('breath_id')['u_out'].shift(-4).fillna(0)
print(len(X_train))

#Window features
#Choice of features inspired vy https://www.kaggle.com/tenffe/finetune-of-tensorflow-bidirectional-lstm
X_train['breath_id__u_in__max'] = X_train.groupby(['breath_id'])['u_in'].transform('max')
X_test['breath_id__u_in__max'] = X_test.groupby(['breath_id'])['u_in'].transform('max')
X_train['breath_id__u_in__mean'] = X_train.groupby(['breath_id'])['u_in'].transform('mean')
X_test['breath_id__u_in__mean'] = X_test.groupby(['breath_id'])['u_in'].transform('mean')
print(len(X_train))

X_train['breath_id__u_out__max'] = X_train.groupby(['breath_id'])['u_out'].transform('max')
X_test['breath_id__u_out__max'] = X_test.groupby(['breath_id'])['u_out'].transform('max')
X_train['breath_id__u_out__mean'] = X_train.groupby(['breath_id'])['u_out'].transform('mean')
X_test['breath_id__u_out__mean'] = X_test.groupby(['breath_id'])['u_out'].transform('mean')

X_train['R'] = X_train['R'].astype(str)
X_train['C'] = X_train['C'].astype(str)
X_train['RC'] = X_train['R']+X_train['C']
X_test['R'] = X_test['R'].astype(str)
X_test['C'] = X_test['C'].astype(str)
X_test['RC'] = X_test['R']+"_"+X_test['C']
print(len(X_train))

X_train = X_train.drop(['R','C','breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same','u_out_lag2'], axis = 1)
X_test = X_test.drop(['R','C','breath_id','one','count','breath_id_lag','breath_id_lag2','breath_id_lagsame','breath_id_lag2same','u_out_lag2'], axis = 1)

print(len(X_train))

X_train

In [ ]:
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)


In [ ]:
Y_train

In [ ]:
scaler = RobustScaler()

X_train[X_train.columns] = scaler.fit_transform(X_train[X_train.columns])
X_test[X_test.columns] = scaler.transform(X_test[X_test.columns])
print(len(X_train))

In [ ]:
len(X_train)

In [ ]:

print(len(X_train))
# Y_train = Y_train.values.reshape(-1,160)
X_train = X_train.values.reshape(-1,80,27+4) # Reshaping into the 3 dimensional tensor that is needed for a recurrent neural network
# X_test = X_test.values.reshape(-1,160,27+4)

In [ ]:
X_train[0][0][0]

In [ ]:
def BidirLSTM(units):
    return layers.Bidirectional(keras.layers.LSTM(units, return_sequences=True))

In [ ]:
def BidirGRU(units):
    return layers.Bidirectional(keras.layers.GRU(units, return_sequences=True, dropout=0.00004))

In [ ]:
def make_lstm_model():
    return keras.models.Sequential([
            layers.Input(shape = (160,27+4)),
            BidirGRU(500),
            BidirGRU(400),
            BidirGRU(300),
            BidirGRU(200),
            layers.Dense(200, activation='swish'),
            layers.Dense(1, activation = 'linear')
        ])
    

In [ ]:
# model = make_lstm_model()
# model.compile(loss='mae', optimizer = keras.optimizers.Adam(learning_rate=0.002),
#                       metrics=[keras.metrics.MeanAbsoluteError()])

In [ ]:

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,
    min_delta=0.0000001,
    restore_best_weights=True,
)

plateau = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor = 0.75,                                     
    patience = 20,                                   
    min_delt = 0.0000001,                                
    cooldown = 0,                               
    verbose = 1
)

In [ ]:
def get_hardware_strategy():
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        tf.config.optimizer.set_jit(True)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    return tpu, strategy

tpu, strategy = get_hardware_strategy()

In [ ]:
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
# tf.config.optimizer.set_jit(True)

with strategy.scope():
    kfold = KFold(n_splits = 2, shuffle=True, random_state = 42)

    pred = np.zeros((len(X_test),1))
    #train_oof = np.zeros((train_len,1))
    train_oof = np.zeros((len(X_train),160,1))

    for fold, idx in enumerate(kfold.split(X=X_train, y=Y_train)):
        
        print(20*'--','Fold: ', fold+1, 20*'--')
        
        train_idx, val_idx = idx[0], idx[1]
        xtrain = X_train[train_idx]
        ytrain = Y_train[train_idx]
        xval = X_train[val_idx]
        yval = Y_train[val_idx]

#         model = new_model()
        model = make_lstm_model()
        model.compile(loss='mae', optimizer = keras.optimizers.Adam(learning_rate=0.002),
                      metrics=[keras.metrics.MeanAbsoluteError()])
        


        model.fit(xtrain, ytrain,
        batch_size = 1024*6, epochs = 1000,
        validation_data=(xval, yval), callbacks=[plateau])
#         validation_data=(xval, yval), callbacks=[early_stopping, plateau])
#         callbacks=[early_stopping, plateau, check]);

        #create predictions
        if(fold==0):
            pred = model.predict(X_test).reshape(-1,1)/kfold.n_splits
        else:
            pred += model.predict(X_test).reshape(-1,1)/kfold.n_splits     
        
        #print(pred)

        val_pred = model.predict(xval)

        # getting out-of-fold predictions on training set
        train_oof[val_idx] = val_pred

In [ ]:
train_oof = train_oof.reshape(-1,1)
Y_train_reshaped = Y_train.reshape(-1,1)

In [ ]:
print("OOF - MAE: {0:0.6f}".format(mean_absolute_error(Y_train_reshaped,train_oof)))

In [ ]:
train_oof = pd.DataFrame(train_oof, columns = ['pressure'])
train_oof.to_csv('train_oof.csv', index=False)

train_oof

In [ ]:
submission['pressure'] = pred
submission.to_csv("submission.csv", index=False)
submission